In [1]:
! pip3 install  hyperopt

     |████████████████████████████████| 1.6 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 31.4 MB/s eta 0:00:01


In [106]:
import pickle as pkl
import pandas as pd
import numpy as np

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
RANDOM_SEED: int = 42
np.random.seed(RANDOM_SEED)

In [85]:
# Loading train/test datasets and results df
results_df_path = 'model_cmp.csv'
df = pd.read_csv('data/winequality-red.csv')
X, y = df.drop('quality', axis=1), df.quality > 5
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED, test_size=.2, stratify=y)


In [87]:
clf_baseline = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train)
y_pred_baseline = clf_baseline.predict(X_test)
f1_baseline = f1_score(y_test, y_pred_baseline)
f1_baseline

0.8259587020648967

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

In [88]:
def objective_cv(params):
    f1_scores = []
    for train_index, test_index in skf.split(X_train, y_train):
        X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
        model = ExtraTreesClassifier(random_state=42, n_jobs=-1, **params).fit(X_train_cv, y_train_cv)
        score = f1_score(y_val, model.predict(X_val))
        f1_scores.append(score)
    return {'loss': -np.mean(f1_scores), 'status': STATUS_OK}

In [89]:
def objective_fast(params):
    X_train_cv, X_val, y_train_cv, y_val =  train_test_split(X_train, y_train, test_size=.2, random_state=RANDOM_SEED)
    model = ExtraTreesClassifier(random_state=42, n_jobs=-1, **params).fit(X_train_cv, y_train_cv)
    score = f1_score(y_val, model.predict(X_val))
    return {'loss': -score, 'status': STATUS_OK}

In [8]:
from hyperopt import rand

In [9]:
def run_experiment(search_space, budget:int = 10, use_cv: bool =True, method: str = 'random') -> float:
    if use_cv:
        objective = objective_cv
    else:
        objective = objective_fast
    
    if method == 'random':
        method = rand.suggest
    elif method == 'tpe':
        method = tpe.suggest
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=method,
        max_evals=budget)
    
    return space_eval(search_space, best_params), objective(space_eval(search_space, best_params))
    
    

In [107]:
search_space={
    'max_depth': hp.randint('max_depth', 1, 200),           
    'min_samples_split':hp.randint('min_samples_split', 2, 5),   
    'min_samples_leaf':hp.randint('min_samples_leaf', 1, 20),
    'criterion':hp.choice('criterion', ['gini', 'entropy']),
    'max_features':hp.choice('max_features', ['sqrt', 'log2', 0.9]),
    'bootstrap':hp.choice('bootstrap', [True, False]),
    "class_weight": hp.choice('class_weight', ['balanced', 'balanced_subsample'])
}
algorithm=tpe.suggest

In [108]:
params, _ = run_experiment(search_space, 5, use_cv=False, method='tpe')
params

100%|██████████| 5/5 [00:02<00:00,  2.36trial/s, best loss: -0.8112449799196787]


{'bootstrap': False,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 22,
 'max_features': 0.9,
 'min_samples_leaf': 4,
 'min_samples_split': 4}

In [109]:
clf = ExtraTreesClassifier(**params, random_state=42, n_jobs=-1).fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

0.7926829268292682

In [146]:
def optmimize(
    datasets, f1_baseline, budget_list, opt_method_list,
    use_cv=True, early_stopping=True
):
    X_train, X_test, y_train, y_test = datasets
    final_params = {}
    final_scores = {}
    try:
        for budget in budget_list:
            for opt_method in opt_method_list:
                print((budget, opt_method))
                # tune model
                params, _ = run_experiment(search_space, budget, use_cv=use_cv, method=opt_method)
                final_params[(budget, opt_method)] = params

                # evaluate model
                clf = ExtraTreesClassifier(random_state=42, **params, n_jobs=-1).fit(X_train, y_train)
                y_pred_opt = clf.predict(X_test)
                f1 = f1_score(y_test, y_pred_opt)
                final_scores[(budget, opt_method)] = f1
                print(f"F1-score: {f1}")

                # early stopping condition, because we are looking for the smallest time budget
                if early_stopping and f1 > f1_baseline:
                    A = ((y_pred_baseline == y_test) & (y_pred_opt == y_test)).sum()
                    B = ((y_pred_baseline != y_test) & (y_pred_opt == y_test)).sum()
                    C = ((y_pred_baseline == y_test) & (y_pred_opt != y_test)).sum()
                    D = ((y_pred_baseline != y_test) & (y_pred_opt != y_test)).sum()
                    result = mcnemar([[A, B], [C, D]])
                    alpha = 0.05
                    if B + C > 20 and result.pvalue < alpha:
                        print(f"Model with time budget {budget} and {opt_method} optimization algo beat the baseline!")
                        return (budget, opt_method), final_params, final_scores
                    else:
                        print("F1 is better, but not statistically")
    except KeyboardInterrupt:
        print("Interrupted.")
        
    print("No model outperformed the baseline")
    return None, final_params, final_scores

In [116]:
final_params = {}
final_scores = {}

In [117]:
datasets = X_train, X_test, y_train, y_test
best, params, scores = optmimize(
    datasets, f1_baseline, budget_list=[10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000], 
    opt_method_list=['tpe'], use_cv=True)
final_params.update(params)
final_scores.update(scores)

(10, 'tpe')
100%|██████████| 10/10 [00:17<00:00,  1.76s/trial, best loss: -0.8161467853296603]
F1-score: 0.8036253776435045
(20, 'tpe')
100%|██████████| 20/20 [00:23<00:00,  1.16s/trial, best loss: -0.8164002509750485]
F1-score: 0.8221574344023324
(50, 'tpe')
100%|██████████| 50/50 [01:24<00:00,  1.70s/trial, best loss: -0.81801586794]     
F1-score: 0.8214285714285714
(100, 'tpe')
100%|██████████| 100/100 [02:55<00:00,  1.75s/trial, best loss: -0.8215628574225663]
F1-score: 0.8168168168168167
(200, 'tpe')
100%|██████████| 200/200 [05:36<00:00,  1.68s/trial, best loss: -0.82216029844896] 
F1-score: 0.8132530120481928
(500, 'tpe')
 26%|██▌       | 130/500 [03:52<11:00,  1.79s/trial, best loss: -0.8251873077524987]
Interrupted.
No model outperformed the baseline


In [119]:
datasets = X_train, X_test, y_train, y_test
best, params, scores = optmimize(
    datasets, f1_baseline, budget_list=[10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000], 
    opt_method_list=['random'], use_cv=True)
final_params.update(params)
final_scores.update(scores)

(10, 'random')
100%|██████████| 10/10 [00:18<00:00,  1.80s/trial, best loss: -0.82168574877736] 
F1-score: 0.8367952522255192
F1 is better, but not statistically
(20, 'random')
100%|██████████| 20/20 [00:39<00:00,  1.97s/trial, best loss: -0.8197646891820621]
F1-score: 0.8143712574850298
(50, 'random')
100%|██████████| 50/50 [01:30<00:00,  1.80s/trial, best loss: -0.81801586794]     
F1-score: 0.8214285714285714
(100, 'random')
100%|██████████| 100/100 [02:57<00:00,  1.77s/trial, best loss: -0.8251873077524987]
F1-score: 0.8245614035087719
(200, 'random')
100%|██████████| 200/200 [06:20<00:00,  1.90s/trial, best loss: -0.8251873077524987]
F1-score: 0.8245614035087719
(500, 'random')
100%|██████████| 500/500 [16:16<00:00,  1.95s/trial, best loss: -0.82168574877736]  
F1-score: 0.8367952522255192
F1 is better, but not statistically
(1000, 'random')
  1%|          | 10/1000 [00:20<34:38,  2.10s/trial, best loss: -0.8251873077524987]
Interrupted.
No model outperformed the baseline


In [123]:
search_space.update({"n_estimators": hp.randint('n_estimators', 50, 200)})

In [148]:
datasets = X_train, X_test, y_train, y_test
best, params, scores = optmimize(
    datasets, f1_baseline, budget_list=[1000, 2000, 5000, 10000], 
    opt_method_list=['random'], use_cv=True)
final_params.update(params)
final_scores.update(scores)

(1000, 'random')
100%|██████████| 1000/1000 [40:14<00:00,  2.41s/trial, best loss: -0.8257992774739824]
F1-score: 0.8259587020648967
(2000, 'random')
100%|██████████| 2000/2000 [1:15:38<00:00,  2.27s/trial, best loss: -0.8277212997419445]
F1-score: 0.813953488372093
(5000, 'random')
100%|██████████| 5000/5000 [3:03:21<00:00,  2.20s/trial, best loss: -0.8319567569242489]  
F1-score: 0.8224852071005917
(10000, 'random')
 38%|███▊      | 3846/10000 [2:24:06<3:50:35,  2.25s/trial, best loss: -0.8281653094223579]
Interrupted.
No model outperformed the baseline


In [149]:
final_params, final_scores

({(10, 'tpe'): {'bootstrap': True,
   'class_weight': 'balanced',
   'criterion': 'gini',
   'max_depth': 182,
   'max_features': 0.9,
   'min_samples_leaf': 2,
   'min_samples_split': 4},
  (20, 'tpe'): {'bootstrap': False,
   'class_weight': 'balanced_subsample',
   'criterion': 'entropy',
   'max_depth': 150,
   'max_features': 'log2',
   'min_samples_leaf': 1,
   'min_samples_split': 3},
  (50, 'tpe'): {'bootstrap': False,
   'class_weight': 'balanced',
   'criterion': 'gini',
   'max_depth': 94,
   'max_features': 'log2',
   'min_samples_leaf': 1,
   'min_samples_split': 4},
  (100, 'tpe'): {'bootstrap': False,
   'class_weight': 'balanced_subsample',
   'criterion': 'entropy',
   'max_depth': 63,
   'max_features': 0.9,
   'min_samples_leaf': 3,
   'min_samples_split': 3},
  (200, 'tpe'): {'bootstrap': False,
   'class_weight': 'balanced',
   'criterion': 'gini',
   'max_depth': 20,
   'max_features': 'sqrt',
   'min_samples_leaf': 2,
   'min_samples_split': 4},
  (10, 'random'):

In [154]:
params = final_params[(10, 'random')]

clf = ExtraTreesClassifier(**params, random_state=42, n_jobs=-1).fit(X_train, y_train)
y_pred_opt = clf.predict(X_test)
metric = f1_score(y_test, y_pred_opt)
metric

0.8367952522255192

In [141]:
f1_baseline

0.8259587020648967

In [142]:
from statsmodels.stats.contingency_tables import mcnemar

In [176]:
A = ((y_pred_baseline == y_test) & (y_pred_opt == y_test)).sum()
B = ((y_pred_baseline != y_test) & (y_pred_opt == y_test)).sum()
C = ((y_pred_baseline == y_test) & (y_pred_opt != y_test)).sum()
D = ((y_pred_baseline != y_test) & (y_pred_opt != y_test)).sum()

contingency_table_df=pd.DataFrame(data={"nr_correct_clf1":["Yes/Yes","No/Yes"], "nr_incorrect_cl1":["Yes/No","No/No"]}, index=["nr_correct_clf2","nr_incorrect_clf2"])
contingency_table_df.iloc[0,0]=A
contingency_table_df.iloc[0,1]=B
contingency_table_df.iloc[1,0]=C
contingency_table_df.iloc[1,1]=D
contingency_table_df

,nr_correct_clf1,nr_incorrect_cl1
nr_correct_clf2,253,12
nr_incorrect_clf2,8,47


In [177]:
B + C

20

After calculating the Contingency Table, we need to define your hypothesis:
*   H0: both models have the same performance
*   H1: performances of the two models are not equal

In [157]:
result = mcnemar([[A, B], [C, D]])
print(result)
alpha = 0.05
if result.pvalue > alpha:
    print('Same proportions of errors (fail to reject H0)')
else:
    print('Different proportions of errors (reject H0)')

pvalue      0.5034446716308595
statistic   8.0
Same proportions of errors (fail to reject H0)


Lets compare on crossvalidation

In [164]:
from tqdm import tqdm

params_list = [final_params[(10, 'random')], final_params[(500, 'random')], {}]
N_FOLDS: int = 5
    
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

results = np.zeros((len(params_list), N_FOLDS))
cur_fold = 0
np.random.seed(RANDOM_SEED)

for train_index, test_index in skf.split(X, y):
    X_train_cv, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train_cv, y_val = y.iloc[train_index], y.iloc[test_index]
    
    for i, params in tqdm(enumerate(params_list)):
        clf = ExtraTreesClassifier(**params, random_state=42, n_jobs=-1).fit(X_train_cv, y_train_cv)
        score = f1_score(y_val, clf.predict(X_val))
        results[i, cur_fold] = score
    cur_fold += 1

3it [00:00,  6.48it/s]
3it [00:00,  6.96it/s]
3it [00:00,  6.81it/s]
3it [00:00,  6.28it/s]
3it [00:00,  6.07it/s]


In [165]:
from scipy.stats import friedmanchisquare

result = friedmanchisquare(*results.tolist())
print(result)

p = result.pvalue
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

FriedmanchisquareResult(statistic=10.0, pvalue=0.006737946999085468)
Different distributions (reject H0)


In [169]:
results = pd.DataFrame(results)
results

,0,1,2,3,4
0,0.816327,0.848837,0.819767,0.861878,0.782090
1,0.816327,0.848837,0.819767,0.861878,0.782090
2,0.809249,0.842105,0.815476,0.856354,0.752294


In [172]:
ranks = pd.DataFrame()
ranks['r1'] = results.loc[:, 0].rank(ascending=False)
ranks['r2'] = results.loc[:, 1].rank(ascending=False)
ranks['r3'] = results.loc[:, 2].rank(ascending=False)
ranks['r4'] = results.loc[:, 3].rank(ascending=False)
ranks['r5'] = results.loc[:, 4].rank(ascending=False)
ranks['mean'] = ranks.mean(axis=1)
ranks

,r1,r2,r3,r4,r5,mean
0,1.5,1.5,1.5,1.5,1.5,1.5
1,1.5,1.5,1.5,1.5,1.5,1.5
2,3.0,3.0,3.0,3.0,3.0,3.0


As we see first two configurations are better than the third one (baseline).

In [175]:
final_params[(10, 'random')], final_params[(500, 'random')]

({'bootstrap': False,
  'class_weight': 'balanced',
  'criterion': 'entropy',
  'max_depth': 55,
  'max_features': 0.9,
  'min_samples_leaf': 1,
  'min_samples_split': 2},
 {'bootstrap': False,
  'class_weight': 'balanced_subsample',
  'criterion': 'entropy',
  'max_depth': 41,
  'max_features': 0.9,
  'min_samples_leaf': 1,
  'min_samples_split': 2})